<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [4]:
# method most_simmilar
'''your code'''

'your code'

In [5]:
wv_embeddings.most_similar("dog", topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

слово cat не Входит в топ-5 близких слов к слову dog

In [6]:
word_1 = 'cat'
for index, word in enumerate(wv_embeddings.most_similar("dog", topn=100)):
    if word[0] == word_1:
        print(f'слово {word_1} занимает место = {index + 1}')

слово cat занимает место = 26


***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [7]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [94]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    '''your code'''
    
    features = np.zeros([dim], dtype='float32')
    
    i = 0
    words = tokenizer.tokenize(question.lower())
    for word in words:
        if word in embeddings:
            features += embeddings[f'{word}']
            i += 1
    if i:
        return features / i
    else:
        return features

In [14]:
round(question_to_vec('123ывм456', wv_embeddings, tokenizer, dim=200)[2], 2)

['123ывм456']


0.0

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [15]:
# Предложение
question = "I love neural networks"

'''your code'''

'your code'

In [16]:
round(question_to_vec(question, wv_embeddings, tokenizer, dim=200)[2], 2)

['i', 'love', 'neural', 'networks']


-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)



In [17]:
import math

In [18]:
rank_q = 9
K = 10
Hits_10 = 1 if rank_q <= K else 0
DGG_10 = 1 / math.log(1 + rank_q, 2) * Hits_10
round(DGG_10, 1)

0.3

#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



максимум = 1

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [19]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    
    # Подсчитываем количество дубликатов, чей ранг <= k
   
    hits_value = sum(np.array(dup_ranks) <= k) / len(dup_ranks)
    return hits_value

In [122]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [166]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    # Вычисляем сумму для всех релевантных дубликатов
    '''your code'''
    
    dcg_value = (1 / np.log2(1 + np.array(dup_ranks))) * (np.array(dup_ranks) <= k)
    # Делим на общее количество вопросов
    '''your code'''
    dcg_value = np.mean(dcg_value)
    return dcg_value

In [173]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@4 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@4 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [174]:
import pandas as pd

In [175]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [176]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


In [131]:
dup_ranks

[2]

In [132]:
hits_count(dup_ranks, k=47)

1.0

In [133]:
dcg_score(dup_ranks, k=1)

0.0

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [29]:
!unzip stackoverflow_similar_questions.zip

"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Считайте данные.

In [30]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [31]:
validation_data = read_corpus('stackoverflow_similar_questions/data/validation.tsv')

Кол-во строк

In [32]:
len(validation_data)

3760

Размер нескольких первых строк

In [33]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [148]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [149]:
from numpy.linalg import norm

In [150]:
def cosine_sim(v1, v2):
    # v1, v2 (1 x dim)
    return np.array(v1 @ v2.T / norm(v1) / norm(v2))

In [163]:
from sklearn.metrics.pairwise import cosine_similarity

In [165]:
cosine_sim(np.array([1, 2]), np.array([3, 4])), cosine_similarity(np.array([[1, 2]]), np.array([[3, 4]]))

(array(0.98386991), array([[0.98386991]]))

In [151]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    
    cosine_sim_list = []
    question_vec = question_to_vec(question, embeddings, tokenizer, dim=dim)
    for candidate in candidates:
        candidate_vec = question_to_vec(candidate, embeddings, tokenizer, dim=dim)
        cosine_sim_list.append(cosine_sim(question_vec, candidate_vec))
    index_list = range(0, len(candidates))
    abc = zip(cosine_sim_list, index_list, candidates)
    result = sorted(abc, reverse=True)
    result = [tuple([b, c]) for a, b, c in result]
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [152]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [153]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [155]:
from tqdm.notebook import tqdm

In [156]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
wv_ranking

In [172]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.402 | Hits@   1: 0.402
DCG@   5: 0.498 | Hits@   5: 0.582
DCG@  10: 0.519 | Hits@  10: 0.650
DCG@ 100: 0.565 | Hits@ 100: 0.874
DCG@ 500: 0.578 | Hits@ 500: 0.973
DCG@1000: 0.581 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [135]:
train_data = read_corpus('stackoverflow_similar_questions/data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [181]:
' '.join(train_data[111258])

'Determine if the device is a smartphone or tablet? Change imageView params in all cards together'

In [179]:
train_data[3]

['How to insert CookieCollection to CookieContainer?',
 'C# create cookie from string and send it']

In [ ]:
# Создаем общий корпус текстов
corpus = []

'''your code'''

In [192]:
# стоп-слова, preprocess
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopWords = set(stopwords.words('english'))
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\София\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [194]:
def preproc_nltk(text, do_delete_punkt=True, do_normalization=True):
    if do_delete_punkt:
        text = re.sub(f'[{string.punctuation}]', ' ', text)
    if do_normalization:
        res = ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])
    
    else:
        res = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])    
    return res

#### без удаления пунктуации, без нормализации, window=5

In [200]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=False, do_normalization=False).split() for text in train_data]

In [201]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 '?',
 'event',
 'handling',
 'geometries',
 'three.js',
 '?']

In [202]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=5, #'''your code''',                # Размер окна контекста
    min_count=3, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [203]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [204]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.392 | Hits@   1: 0.392
DCG@   5: 0.499 | Hits@   5: 0.590
DCG@  10: 0.523 | Hits@  10: 0.664
DCG@ 100: 0.569 | Hits@ 100: 0.887
DCG@ 500: 0.582 | Hits@ 500: 0.981
DCG@1000: 0.584 | Hits@1000: 1.000


#### без удаления пунктуации, с нормализацией, window=5

In [185]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=False, do_normalization=True).split() for text in train_data]

In [187]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 '?',
 'event',
 'handling',
 'geometry',
 'three.js',
 '?']

In [189]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=5, #'''your code''',                # Размер окна контекста
    min_count=3, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [190]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [191]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.366 | Hits@   1: 0.366
DCG@   5: 0.474 | Hits@   5: 0.569
DCG@  10: 0.497 | Hits@  10: 0.641
DCG@ 100: 0.541 | Hits@ 100: 0.860
DCG@ 500: 0.557 | Hits@ 500: 0.978
DCG@1000: 0.559 | Hits@1000: 1.000


#### c удалением пунктуации, с нормализацией, window=5

In [195]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=True, do_normalization=True).split() for text in train_data]

In [196]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 'event',
 'handling',
 'geometry',
 'three',
 'j']

In [197]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=5, #'''your code''',                # Размер окна контекста
    min_count=3, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [198]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [199]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.380 | Hits@   1: 0.380
DCG@   5: 0.476 | Hits@   5: 0.560
DCG@  10: 0.503 | Hits@  10: 0.644
DCG@ 100: 0.548 | Hits@ 100: 0.865
DCG@ 500: 0.563 | Hits@ 500: 0.978
DCG@1000: 0.566 | Hits@1000: 1.000


#### c удалением пунктуации, с нормализацией, window=10

In [205]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=True, do_normalization=True).split() for text in train_data]

In [206]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 'event',
 'handling',
 'geometry',
 'three',
 'j']

In [207]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=10, #'''your code''',                # Размер окна контекста
    min_count=3, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [208]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [209]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.491 | Hits@   5: 0.572
DCG@  10: 0.518 | Hits@  10: 0.657
DCG@ 100: 0.562 | Hits@ 100: 0.867
DCG@ 500: 0.576 | Hits@ 500: 0.975
DCG@1000: 0.578 | Hits@1000: 1.000


#### c удалением пунктуации, с нормализацией, window=10, min_count=1

In [205]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=True, do_normalization=True).split() for text in train_data]

In [206]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 'event',
 'handling',
 'geometry',
 'three',
 'j']

In [214]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=10, #'''your code''',                # Размер окна контекста
    min_count=1, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [215]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [216]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.389 | Hits@   1: 0.389
DCG@   5: 0.486 | Hits@   5: 0.568
DCG@  10: 0.511 | Hits@  10: 0.646
DCG@ 100: 0.555 | Hits@ 100: 0.860
DCG@ 500: 0.570 | Hits@ 500: 0.974
DCG@1000: 0.572 | Hits@1000: 1.000


#### c удалением пунктуации, с нормализацией, window=20, min_count=3

In [205]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=True, do_normalization=True).split() for text in train_data]

In [206]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 'event',
 'handling',
 'geometry',
 'three',
 'j']

In [210]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=20, #'''your code''',                # Размер окна контекста
    min_count=3, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [211]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [212]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.409 | Hits@   1: 0.409
DCG@   5: 0.506 | Hits@   5: 0.591
DCG@  10: 0.528 | Hits@  10: 0.660
DCG@ 100: 0.571 | Hits@ 100: 0.867
DCG@ 500: 0.585 | Hits@ 500: 0.974
DCG@1000: 0.588 | Hits@1000: 1.000


#### c удалением пунктуации, без нормализации, window=20, min_count=3

In [217]:
corpus = [preproc_nltk(' '.join(text), do_delete_punkt=True, do_normalization=False).split() for text in train_data]

In [218]:
corpus[1]

['html',
 '5',
 'canvas',
 'javascript',
 'use',
 'making',
 'interactive',
 'drawing',
 'tool',
 'event',
 'handling',
 'geometries',
 'three',
 'js']

In [219]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=20, #'''your code''',                # Размер окна контекста
    min_count=3, #'''your code''',             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [220]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

C:\Users\София\AppData\Local\Temp\ipykernel_13592\461196727.py:3: RuntimeWarning: invalid value encountered in float_scalars
  return np.array(v1 @ v2.T / norm(v1) / norm(v2))


In [221]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.430 | Hits@   1: 0.430
DCG@   5: 0.534 | Hits@   5: 0.621
DCG@  10: 0.555 | Hits@  10: 0.687
DCG@ 100: 0.599 | Hits@ 100: 0.896
DCG@ 500: 0.610 | Hits@ 500: 0.980
DCG@1000: 0.612 | Hits@1000: 1.000


In [213]:
np.mean([len(a) for a in corpus])

13.133689

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


- лучше всего с удалением пунктуации, без нормализации
- чем больше контекстное окно, тем лучше

- Какой принцип токенизации даёт качество лучше и почему? - токенизация с учетом удаления пунктуации
- Помогает ли нормализация слов? - нет, ухудшает
- Какие эмбеддинги лучше справляются с задачей и почему? - дообученные
- Почему получилось плохое качество решения задачи? - возможно надо пробовать fasttext
- Предложите свой подход к решению задачи. - fasttext / bert